In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# 🏈 Predicting Blitzes Using Pre-Snap Behavior

**Authors:**  
- Chris Doyle (christopherdoyle@college.harvard.edu)  
- Hans Elasri (hanselasri@college.harvard.edu)  
- Thomas Garity (tgarity@college.harvard.edu)  
- Rishi Hazra (rishihazra@college.harvard.edu)  
- Chris Ruaño (cruano@college.harvard.edu)

---

## Project Summary

Blitzing is one of the most aggressive and high-risk strategies in football. When executed well, it can disrupt an offensive drive by forcing the quarterback into rushed decisions; when misread or mistimed, it can leave the defense vulnerable to big plays. Offensive coaches and quarterbacks spend countless hours studying pre-snap cues to anticipate incoming blitzes, while defenses work just as hard to disguise them through subtle shifts, delayed rushes, and simulated pressures.

Our project aims to bring analytics into this equation by predicting whether a defense will blitz, using only pre-snap player tracking data from the NFL Big Data Bowl 2025 dataset. By extracting features such as player positioning, movement trends, alignment depth, and formation structure, we seek to develop machine learning models that systematically classify plays as blitz or non-blitz scenarios. 

A successful model would not only help identify the most telling pre-snap indicators of pressure but also provide a practical tool for offensive strategists to better anticipate and counter defensive blitzes, enhancing both game preparation and real-time decision-making.

---

## Data Structure

Our data can be downloaded from the 2025 Big Data Bowl on [Kaggle](https://www.kaggle.com/competitions/nfl-big-data-bowl-2025/data)  
or using the Kaggle API:

```
kaggle competitions download -c nfl-big-data-bowl-2025
```

**Instructions:**
- Download the dataset and unzip the file `nfl-big-data-bowl-2025.zip`.
- The datasets will be saved in a `\nfl-big-data-bowl-2025` directory within the project repo.
- Ensure that this directory is listed in your `.gitignore` file to avoid pushing large data files to GitHub.

Now, let's check that the data has been downloaded correctly:

In [2]:
# ensure all tables downloaded
print("Data Availabe: ")
os.listdir('./nfl-big-data-bowl-2025/')

Data Availabe: 


['plays.csv',
 'tracking_week_5.csv',
 'tracking_week_4.csv',
 'tracking_week_6.csv',
 'tracking_week_7.csv',
 'tracking_week_3.csv',
 'tracking_week_2.csv',
 'tracking_week_1.csv',
 'players.csv',
 'games.csv',
 'tracking_week_9.csv',
 'tracking_week_8.csv',
 'player_play.csv']

In [3]:
!du -sh nfl-big-data-bowl-2025

7.6G	nfl-big-data-bowl-2025


In [4]:
plays_df = pd.read_csv('./nfl-big-data-bowl-2025/plays.csv')
players_df = pd.read_csv('./nfl-big-data-bowl-2025/players.csv')
games_df = pd.read_csv('./nfl-big-data-bowl-2025/games.csv')
player_play_df = pd.read_csv('./nfl-big-data-bowl-2025/player_play.csv')
tracking_weeks = pd.DataFrame()
for week in range(1, 10):
    current_week = pd.read_csv(f'./nfl-big-data-bowl-2025/tracking_week_{week}.csv')
    tracking_weeks = pd.concat([tracking_weeks, current_week], ignore_index=True)


## Data Summary

### 1. `games.csv`
- **Purpose:** Info about each game.
- **Important Variables:**
  - `gameId` (primary key)
  - `season`, `week`
  - `homeTeamAbbr`, `visitorTeamAbbr`
- **Project Relevance:** Mostly for joining, basic game context (e.g., week, matchup). Not critical for blitz prediction itself.

---

### 2. `plays.csv`
- **Purpose:** Play-level metadata.
- **Important Variables:**
  - `gameId`, `playId` (keys)
  - `quarter`, `down`, `yardsToGo`
  - `possessionTeam`, `defensiveTeam`
  - `offenseFormation`
  - `playDescription`
  - `isDropback` (Boolean: did QB drop back)
  - `pff_passCoverage` (type of defensive coverage)
  - `pff_manZone` (man vs zone coverage)
  - `playAction` (play-action pass or not)
- **Project Relevance:** Crucial for understanding pre-snap situation and defensive alignment; used for labels or features in blitz prediction.

---

### 3. `players.csv`
- **Purpose:** Static player information.
- **Important Variables:**
  - `nflId` (player key)
  - `position`
  - `displayName`
- **Project Relevance:** Helpful for interpreting player roles; minor for pure blitz prediction unless modeling specific player tendencies.

---

### 4. `player_play.csv`
- **Purpose:** Player-level stats per play.
- **Important Variables:**
  - `gameId`, `playId`, `nflId` (keys)
  - `teamAbbr`
  - `wasInitialPassRusher` (binary, key for identifying blitzers!)
  - `causedPressure`
  - `timeToPressureAsPassRusher`
  - `getOffAsPassRusher`
- **Project Relevance:** Allows you to know who blitzed (extra rushers) and pressure dynamics.

---

### 5. `tracking_week_[1-9].csv`
- **Purpose:** Frame-by-frame tracking data (player movement).
- **Important Variables:**
  - `gameId`, `playId`, `nflId`
  - `frameId`, `time`
  - `x`, `y` (player positions)
  - `s`, `a`, `dis` (speed, acceleration, distance moved)
  - `o` (orientation) and `dir` (direction)
  - `event` (snap, ball release, etc.)
- **Project Relevance:** Used to generate **pre-snap features** like player alignments, movement, speed, and timing at snap. Core inputs for neural networks.


In [5]:
plays_df.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber',
       'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore',
       'playNullifiedByPenalty', 'absoluteYardlineNumber',
       'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability',
       'expectedPoints', 'offenseFormation', 'receiverAlignment',
       'playClockAtSnap', 'passResult', 'passLength', 'targetX', 'targetY',
       'playAction', 'dropbackType', 'dropbackDistance', 'passLocationType',
       'timeToThrow', 'timeInTackleBox', 'timeToSack', 'passTippedAtLine',
       'unblockedPressure', 'qbSpike', 'qbKneel', 'qbSneak',
       'rushLocationType', 'penaltyYards', 'prePenaltyYardsGained',
       'yardsGained', 'homeTeamWinProbabilityAdded',
       'visitorTeamWinProbilityAdded', 'expectedPointsAdded', 'isDropback',
       'pff_runConceptPrimary', 'pff_runConceptSecondary', 'pff_runPassOption',
       'pff_pass

In [6]:
plays_df = plays_df.drop(columns=['playDescription', 'playNullifiedByPenalty', 'yardlineSide', 'yardlineNumber', 'playClockAtSnap', 'passResult',
                                    'passLength', 'targetX', 'targetY', 'playAction', 'dropbackType', 'dropbackDistance', 'passLocationType',
                                    'timeToThrow', 'timeInTackleBox', 'timeToSack', 'passTippedAtLine',
                                    'unblockedPressure', 'qbSpike', 'qbKneel', 'qbSneak',
                                    'rushLocationType', 'penaltyYards', 'prePenaltyYardsGained',
                                    'yardsGained', 'homeTeamWinProbabilityAdded',
                                    'visitorTeamWinProbilityAdded', 'expectedPointsAdded', 'isDropback',
                                    'pff_runConceptPrimary', 'pff_runConceptSecondary', 'pff_runPassOption',
                                    'pff_passCoverage', 'pff_manZone'])
plays_df['offenseFormation'] = plays_df['offenseFormation'].fillna('EMPTY')
# mm:ss convert to minutes in column gameClock
def convert_game_clock_to_minutes(game_clock):
    if pd.isna(game_clock):
        return np.nan
    minutes, seconds = map(int, game_clock.split(':'))
    return minutes + seconds / 60
plays_df['gameClock'] = plays_df['gameClock'].apply(convert_game_clock_to_minutes)
plays_df.head()

,gameId,playId,quarter,down,yardsToGo,possessionTeam,defensiveTeam,gameClock,preSnapHomeScore,preSnapVisitorScore,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment
0,2022102302,2655,3,1,10,CIN,ATL,1.900000,35,17,31,0.982017,0.017983,0.719313,EMPTY,3x2
1,2022091809,3698,4,1,10,CIN,DAL,2.216667,17,17,18,0.424356,0.575644,0.607746,EMPTY,3x2
2,2022103004,3146,4,3,12,HOU,TEN,2.000000,3,17,30,0.006291,0.993709,-0.291485,SHOTGUN,2x2
3,2022110610,348,1,2,10,KC,TEN,9.466667,0,0,33,0.884223,0.115777,4.249382,SHOTGUN,2x2
4,2022102700,2799,3,2,8,BAL,TB,2.266667,10,10,37,0.410371,0.589629,3.928413,PISTOL,3x1


In [7]:
# convert height to inches (current format: '6-4' -> 76)
def convert_height_to_inches(height):
    if pd.isna(height):
        return np.nan
    feet, inches = map(int, height.split('-'))
    return feet * 12 + inches
players_df['height'] = players_df['height'].apply(convert_height_to_inches)

players_df = players_df.drop(columns=['birthDate', 'collegeName', 'displayName'])
players_df['nflId'] = players_df['nflId'].astype(str)
players_df.head()

,nflId,height,weight,position
0,25511,76,225,QB
1,29550,76,328,T
2,29851,74,225,QB
3,30842,78,267,TE
4,33084,76,217,QB


In [8]:
games_df = games_df.drop(columns=['season', 'gameDate', 'gameTimeEastern', 'homeFinalScore', 'visitorFinalScore'])
games_df.head()

,gameId,week,homeTeamAbbr,visitorTeamAbbr
0,2022090800,1,LA,BUF
1,2022091100,1,ATL,NO
2,2022091101,1,CAR,CLE
3,2022091102,1,CHI,SF
4,2022091103,1,CIN,PIT


In [9]:
# set nflId as string
player_play_df['nflId'] = player_play_df['nflId'].astype(str)
player_play_df.head()

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,...,wasRunningRoute,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
0,2022090800,56,35472,BUF,0,0,0,0,0,0,...,NaN,NaN,47917.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2022090800,56,42392,BUF,0,0,0,0,0,0,...,NaN,NaN,47917.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2022090800,56,42489,BUF,0,0,0,0,0,1,...,1.0,IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022090800,56,44875,BUF,0,0,0,0,0,0,...,NaN,NaN,43335.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,2022090800,56,44985,BUF,0,0,0,0,0,0,...,1.0,OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
player_play_df.columns

Index(['gameId', 'playId', 'nflId', 'teamAbbr', 'hadRushAttempt',
       'rushingYards', 'hadDropback', 'passingYards', 'sackYardsAsOffense',
       'hadPassReception', 'receivingYards', 'wasTargettedReceiver',
       'yardageGainedAfterTheCatch', 'fumbles', 'fumbleLost',
       'fumbleOutOfBounds', 'assistedTackle', 'forcedFumbleAsDefense',
       'halfSackYardsAsDefense', 'passDefensed', 'quarterbackHit',
       'sackYardsAsDefense', 'safetyAsDefense', 'soloTackle', 'tackleAssist',
       'tackleForALoss', 'tackleForALossYardage', 'hadInterception',
       'interceptionYards', 'fumbleRecoveries', 'fumbleRecoveryYards',
       'penaltyYards', 'penaltyNames', 'wasInitialPassRusher',
       'causedPressure', 'timeToPressureAsPassRusher',
       'getOffTimeAsPassRusher', 'inMotionAtBallSnap', 'shiftSinceLineset',
       'motionSinceLineset', 'wasRunningRoute', 'routeRan',
       'blockedPlayerNFLId1', 'blockedPlayerNFLId2', 'blockedPlayerNFLId3',
       'pressureAllowedAsBlocker', 'timeT

In [11]:
player_play_df = player_play_df[['gameId', 'playId', 'nflId', 'teamAbbr', 'hadRushAttempt',
                                    'rushingYards', 'hadDropback', 'passingYards',
                                    'hadPassReception', 'receivingYards',
                                    'fumbles', 'forcedFumbleAsDefense',
                                    'quarterbackHit',
                                    'sackYardsAsDefense', 'soloTackle',
                                    'tackleForALoss', 'tackleForALossYardage', 'hadInterception',
                                    'interceptionYards', 'fumbleRecoveries',
                                    'penaltyYards', 'penaltyNames', 'wasInitialPassRusher',
                                    'causedPressure', 'timeToPressureAsPassRusher',
                                    'getOffTimeAsPassRusher', 'shiftSinceLineset',
                                    'motionSinceLineset',
                                    'blockedPlayerNFLId1', 'blockedPlayerNFLId2', 'blockedPlayerNFLId3',
                                    'pressureAllowedAsBlocker', 'timeToPressureAllowedAsBlocker']]
# set nlfId as string
player_play_df['nflId'] = player_play_df['nflId'].astype(str)
player_play_df.head()

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,hadPassReception,receivingYards,...,causedPressure,timeToPressureAsPassRusher,getOffTimeAsPassRusher,shiftSinceLineset,motionSinceLineset,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker
0,2022090800,56,35472,BUF,0,0,0,0,0,0,...,False,NaN,NaN,False,False,47917.0,NaN,NaN,0.0,NaN
1,2022090800,56,42392,BUF,0,0,0,0,0,0,...,False,NaN,NaN,False,False,47917.0,NaN,NaN,0.0,NaN
2,2022090800,56,42489,BUF,0,0,0,0,1,6,...,False,NaN,NaN,False,True,NaN,NaN,NaN,NaN,NaN
3,2022090800,56,44875,BUF,0,0,0,0,0,0,...,False,NaN,NaN,False,False,43335.0,NaN,NaN,0.0,NaN
4,2022090800,56,44985,BUF,0,0,0,0,0,0,...,False,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN


In [12]:
tracking_weeks.head()
# set nlfID as string after replacing ".0" with ""
tracking_weeks['nflId'] = tracking_weeks['nflId'].astype(str).str.replace('.0', '', regex=False)


In [13]:
tracking_weeks.columns

Index(['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'frameType',
       'time', 'jerseyNumber', 'club', 'playDirection', 'x', 'y', 's', 'a',
       'dis', 'o', 'dir', 'event'],
      dtype='object')

In [ ]:
tracking_weeks = tracking_weeks[['gameId', 'playId', 'nflId', 'frameId', 'frameType',
                                    'playDirection', 'x', 'y', 's', 'a',
                                    'dis', 'o', 'dir']]
# replace ".0" in nflid with ""
tracking_weeks['nflId'] = tracking_weeks['nflId'].astype(str).str.replace('.0', '', regex=False)
tracking_weeks['nflId'] = tracking_weeks['nflId'].astype(object)
tracking_weeks.head()

We utilize only BEFORE_SNAP data, as it represents the complete set of information accessible when the decision to blitz must be made during actual gameplay.

In [ ]:
tracking_weeks_filtered = tracking_weeks[tracking_weeks['frameType'] == 'BEFORE_SNAP' ]
tracking_weeks_filtered.head()

,gameId,playId,nflId,frameId,frameType,playDirection,x,y,s,a,dis,o,dir
0,2022091200,64,35459,1,BEFORE_SNAP,right,51.06,28.55,0.72,0.37,0.07,246.17,68.34
1,2022091200,64,35459,2,BEFORE_SNAP,right,51.13,28.57,0.71,0.36,0.07,245.41,71.21
2,2022091200,64,35459,3,BEFORE_SNAP,right,51.20,28.59,0.69,0.23,0.07,244.45,69.90
3,2022091200,64,35459,4,BEFORE_SNAP,right,51.26,28.62,0.67,0.22,0.07,244.45,67.98
4,2022091200,64,35459,5,BEFORE_SNAP,right,51.32,28.65,0.65,0.34,0.07,245.74,62.83


In [ ]:
master_df = pd.DataFrame()

In [ ]:
# merge player_play_df with players_df to get player names
master_df = player_play_df.merge(players_df[['nflId', 'position']], on='nflId', how='left')
master_df.head()

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,hadPassReception,receivingYards,...,timeToPressureAsPassRusher,getOffTimeAsPassRusher,shiftSinceLineset,motionSinceLineset,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,position
0,2022090800,56,35472,BUF,0,0,0,0,0,0,...,NaN,NaN,False,False,47917.0,NaN,NaN,0.0,NaN,G
1,2022090800,56,42392,BUF,0,0,0,0,0,0,...,NaN,NaN,False,False,47917.0,NaN,NaN,0.0,NaN,C
2,2022090800,56,42489,BUF,0,0,0,0,1,6,...,NaN,NaN,False,True,NaN,NaN,NaN,NaN,NaN,WR
3,2022090800,56,44875,BUF,0,0,0,0,0,0,...,NaN,NaN,False,False,43335.0,NaN,NaN,0.0,NaN,T
4,2022090800,56,44985,BUF,0,0,0,0,0,0,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,WR


In [ ]:
old_master_df = master_df.copy()
# set nflId to data type object
old_master_df['nflId'] = old_master_df['nflId'].astype(object)

In [ ]:
# flatten rows with same gameid and playid by creating new columns per attribute per player
attr_cols = (
    player_play_df.columns[2:].tolist()   # everything after gameId, playId
  + tracking_weeks.columns[3:].tolist()   # everything after gameId, playId, nflID
  + players_df.columns[1:].tolist()       # everything after nflId
)

attr_cols = [c for c in dict.fromkeys(attr_cols) if c in master_df.columns]
master_df['player_slot'] = (
    master_df
      .sort_values(['gameId','playId','nflId'])    # or whatever ordering you prefer
      .groupby(['gameId','playId'])
      .cumcount()
    + 1
)

wide = (
    master_df
      .set_index(['gameId','playId','player_slot'])[attr_cols]
      .unstack('player_slot')
)

wide.columns = [
    f'{attr}_p{slot}'
    for attr, slot in wide.columns
]

wide = wide.reset_index()

master_df = wide

In [ ]:
master_df.head()

,gameId,playId,nflId_p1,nflId_p2,nflId_p3,nflId_p4,nflId_p5,nflId_p6,nflId_p7,nflId_p8,...,position_p13,position_p14,position_p15,position_p16,position_p17,position_p18,position_p19,position_p20,position_p21,position_p22
0,2022090800,56,35472,38577,41239,42392,42489,42816,43294,43298,...,SS,RB,TE,NT,SS,C,WR,FS,T,ILB
1,2022090800,80,35472,38577,41239,42392,42489,42816,43294,43298,...,SS,RB,TE,NT,SS,C,WR,FS,T,ILB
2,2022090800,101,35472,38577,41239,42392,42816,42818,43294,43298,...,RB,CB,TE,NT,OLB,SS,C,WR,FB,T
3,2022090800,122,35472,38577,41239,42392,42489,42816,43294,43298,...,SS,RB,CB,TE,NT,OLB,SS,C,WR,T
4,2022090800,167,35472,38577,41239,42392,42489,42816,43294,43298,...,SS,CB,TE,NT,OLB,SS,C,RB,WR,T


In [ ]:
# merge player_play_df with plays_df on both playId and gameId
master_df = master_df.merge(plays_df, on=['playId', 'gameId'], how='left')
master_df.head()

,gameId,playId,nflId_p1,nflId_p2,nflId_p3,nflId_p4,nflId_p5,nflId_p6,nflId_p7,nflId_p8,...,defensiveTeam,gameClock,preSnapHomeScore,preSnapVisitorScore,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment
0,2022090800,56,35472,38577,41239,42392,42489,42816,43294,43298,...,LA,15.000000,0,0,85,0.413347,0.586653,1.298699,SHOTGUN,2x2
1,2022090800,80,35472,38577,41239,42392,42489,42816,43294,43298,...,LA,14.483333,0,0,79,0.413316,0.586684,1.303119,EMPTY,3x2
2,2022090800,101,35472,38577,41239,42392,42816,42818,43294,43298,...,LA,13.900000,0,0,72,0.399819,0.600181,2.126690,I_FORM,2x1
3,2022090800,122,35472,38577,41239,42392,42489,42816,43294,43298,...,LA,13.250000,0,0,65,0.384969,0.615031,2.689053,SHOTGUN,2x1
4,2022090800,167,35472,38577,41239,42392,42489,42816,43294,43298,...,LA,11.900000,0,0,57,0.387554,0.612446,2.574206,EMPTY,3x2


In [ ]:
# merge master_df with games_df on gameId
master_df = master_df.merge(games_df, on='gameId', how='left')
master_df.head()

,gameId,playId,nflId_p1,nflId_p2,nflId_p3,nflId_p4,nflId_p5,nflId_p6,nflId_p7,nflId_p8,...,preSnapVisitorScore,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment,week,homeTeamAbbr,visitorTeamAbbr
0,2022090800,56,35472,38577,41239,42392,42489,42816,43294,43298,...,0,85,0.413347,0.586653,1.298699,SHOTGUN,2x2,1,LA,BUF
1,2022090800,80,35472,38577,41239,42392,42489,42816,43294,43298,...,0,79,0.413316,0.586684,1.303119,EMPTY,3x2,1,LA,BUF
2,2022090800,101,35472,38577,41239,42392,42816,42818,43294,43298,...,0,72,0.399819,0.600181,2.126690,I_FORM,2x1,1,LA,BUF
3,2022090800,122,35472,38577,41239,42392,42489,42816,43294,43298,...,0,65,0.384969,0.615031,2.689053,SHOTGUN,2x1,1,LA,BUF
4,2022090800,167,35472,38577,41239,42392,42489,42816,43294,43298,...,0,57,0.387554,0.612446,2.574206,EMPTY,3x2,1,LA,BUF


In [ ]:
nontime_data = master_df.copy()

In [ ]:
nontime_data.head()

,gameId,playId,nflId_p1,nflId_p2,nflId_p3,nflId_p4,nflId_p5,nflId_p6,nflId_p7,nflId_p8,...,preSnapVisitorScore,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment,week,homeTeamAbbr,visitorTeamAbbr
0,2022090800,56,35472,38577,41239,42392,42489,42816,43294,43298,...,0,85,0.413347,0.586653,1.298699,SHOTGUN,2x2,1,LA,BUF
1,2022090800,80,35472,38577,41239,42392,42489,42816,43294,43298,...,0,79,0.413316,0.586684,1.303119,EMPTY,3x2,1,LA,BUF
2,2022090800,101,35472,38577,41239,42392,42816,42818,43294,43298,...,0,72,0.399819,0.600181,2.126690,I_FORM,2x1,1,LA,BUF
3,2022090800,122,35472,38577,41239,42392,42489,42816,43294,43298,...,0,65,0.384969,0.615031,2.689053,SHOTGUN,2x1,1,LA,BUF
4,2022090800,167,35472,38577,41239,42392,42489,42816,43294,43298,...,0,57,0.387554,0.612446,2.574206,EMPTY,3x2,1,LA,BUF


In [ ]:
old_master_df.head()

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,hadPassReception,receivingYards,...,timeToPressureAsPassRusher,getOffTimeAsPassRusher,shiftSinceLineset,motionSinceLineset,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,position
0,2022090800,56,35472,BUF,0,0,0,0,0,0,...,NaN,NaN,False,False,47917.0,NaN,NaN,0.0,NaN,G
1,2022090800,56,42392,BUF,0,0,0,0,0,0,...,NaN,NaN,False,False,47917.0,NaN,NaN,0.0,NaN,C
2,2022090800,56,42489,BUF,0,0,0,0,1,6,...,NaN,NaN,False,True,NaN,NaN,NaN,NaN,NaN,WR
3,2022090800,56,44875,BUF,0,0,0,0,0,0,...,NaN,NaN,False,False,43335.0,NaN,NaN,0.0,NaN,T
4,2022090800,56,44985,BUF,0,0,0,0,0,0,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,WR


In [ ]:
# merge master_df with tracking_weeks on gameId, playId, and nflId
master_df = old_master_df.merge(tracking_weeks, on=['gameId', 'playId', 'nflId'], how='left')
master_df.head()

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,hadPassReception,receivingYards,...,frameId,frameType,playDirection,x,y,s,a,dis,o,dir
0,2022090800,56,35472,BUF,0,0,0,0,0,0,...,1,BEFORE_SNAP,left,89.48,29.52,0.68,1.17,0.07,308.30,266.54
1,2022090800,56,35472,BUF,0,0,0,0,0,0,...,2,BEFORE_SNAP,left,89.39,29.51,0.90,1.33,0.09,301.21,268.31
2,2022090800,56,35472,BUF,0,0,0,0,0,0,...,3,BEFORE_SNAP,left,89.28,29.52,1.13,1.29,0.11,294.13,271.96
3,2022090800,56,35472,BUF,0,0,0,0,0,0,...,4,BEFORE_SNAP,left,89.15,29.52,1.26,0.96,0.12,293.23,271.65
4,2022090800,56,35472,BUF,0,0,0,0,0,0,...,5,BEFORE_SNAP,left,89.00,29.51,1.47,0.88,0.15,271.60,269.37


In [ ]:
# flatten rows with same gameid and playid by creating new columns per attribute per player
attr_cols = (
    player_play_df.columns[2:].tolist()   # everything after gameId, playId
  + tracking_weeks.columns[4:].tolist()   # everything after gameId, playId, nflID, and frameID
  + players_df.columns[1:].tolist()       # everything after nflId
)

attr_cols = [c for c in dict.fromkeys(attr_cols) if c in master_df.columns]
master_df['player_slot'] = (
    master_df
      .sort_values(['gameId','playId', 'nflId'])
      .groupby(['gameId','playId'])
      .cumcount()
    + 1
)

wide = (
    master_df
      .set_index(['gameId','playId','player_slot'])[attr_cols]
      .unstack('player_slot')
)

wide.columns = [
    f'{attr}_p{slot}'
    for attr, slot in wide.columns
]

wide = wide.reset_index()

master_df = wide

: 

In [ ]:
master_df.head()

## Blitz Labeling Methodology

To label whether a play is a blitz, we adopt a method inspired by Dominic Borsani’s approach in the 2023 NFL Big Data Bowl. Rather than relying directly on provided scouting labels, we infer blitz likelihood based on **pre-snap defender behavior** observable in the tracking data.

Specifically, we use the following steps:

- Merge tracking data (`tracking_week_*`) with play metadata from `plays.csv` to access line of scrimmage (`LOS`) information.
- Identify **frames where `frameType == 'SNAP'`**, which correspond to the official ball snap moment.
- For each defender at ball snap:
  - **Distance from LOS**: Compute the absolute difference between the player’s `x` coordinate and the line of scrimmage (`LOS_x`), adjusted for play direction.
  - **Motion Toward LOS**: Estimate whether a player is moving toward the ball using the `dir` (direction) and `playDirection`.
  - **Velocity Toward LOS**: Calculate as `speed × motion_toward_ball`.
- A defender is flagged as a **likely blitzer** if:
  - They are **within 5 yards** of the line of scrimmage at the snap, **and**
  - They are **moving toward the LOS at greater than 1.5 yards per second**.
- At the play level, we sum the number of players flagged as likely blitzers.

We define a **blitz play** as one where **more than 1 defender** meets the blitz criteria at the snap.  
This threshold aligns with the intuition that sending multiple extra rushers constitutes a true blitz rather than just a normal pass rush.

---

### Reference

Borsani, D. (2023). *Beat the Offensive Line: Using Data to Determine Blitz Strategy.* NFL Big Data Bowl, Finalist Project.


In [ ]:
# count unique plays
unique_plays = tracking_weeks[['gameId', 'playId']].drop_duplicates()
print(f"Number of unique plays: {len(unique_plays)}")

In [ ]:
plays_subset = plays_df[['gameId', 'playId', 'yardlineNumber', 'yardlineSide', 'absoluteYardlineNumber']]
tracking = tracking_weeks.merge(plays_subset, on=['gameId', 'playId'], how='left')

# keep only frames right at ball snap
snap_tracking = tracking[tracking['frameType'] == 'SNAP']

# calculate LOS x-coordinate
def calculate_los_x(playDirection, absoluteYardlineNumber):
    if playDirection == 'left':
        return 100 - absoluteYardlineNumber
    else:
        return absoluteYardlineNumber

snap_tracking['LOS_x'] = snap_tracking.apply(
    lambda row: calculate_los_x(row['playDirection'], row['absoluteYardlineNumber']), axis=1
)

# distance from LOS
snap_tracking['dist_from_LOS'] = np.abs(snap_tracking['x'] - snap_tracking['LOS_x'])

# motion toward ball based on play direction
def motion_toward_ball(row):
    if row['playDirection'] == 'left':
        return np.cos(np.deg2rad(row['dir']))
    else:
        return np.cos(np.deg2rad(row['dir'] - 180))

snap_tracking['motion_toward_ball'] = snap_tracking.apply(motion_toward_ball, axis=1)

# velocity toward LOS
snap_tracking['velocity_toward_LOS'] = snap_tracking['s'] * snap_tracking['motion_toward_ball']

# likely blitzer = close to LOS and moving fast toward ball
snap_tracking['isLikelyBlitzer'] = (
    (snap_tracking['dist_from_LOS'] <= 5) &
    (snap_tracking['velocity_toward_LOS'] > 1.5)
).astype(int)

# count likely blitzers
num_likely_blitzers = snap_tracking['isLikelyBlitzer'].sum()
print(f"Total number of players flagged as likely blitzers: {num_likely_blitzers}")

# count blitzers per play
blitzer_counts_by_play = snap_tracking.groupby(['gameId', 'playId'])['isLikelyBlitzer'].sum().reset_index()
print(blitzer_counts_by_play.head())

In [ ]:
# count plays with 1+ blitzer
plays_with_blitzers = blitzer_counts_by_play[blitzer_counts_by_play['isLikelyBlitzer'] > 0]
num_plays_with_blitzers = len(plays_with_blitzers)

# Count total unique plays
total_plays = len(blitzer_counts_by_play)

# calculate percentage
blitz_percentage = (num_plays_with_blitzers / total_plays) * 100 if total_plays > 0 else 0

print(f"Total plays: {total_plays}")
print(f"Plays with at least one blitzer: {num_plays_with_blitzers}")
print(f"Percentage of plays with blitzers: {blitz_percentage:.2f}%")

# count pass plays
pass_plays = plays_df[plays_df['isDropback'] == 1]
print(f"Number of pass plays: {len(pass_plays)}")

# calculate percentage of pass plays with blitzers
pass_plays_with_blitzers = pass_plays[pass_plays['playId'].isin(plays_with_blitzers['playId'])]
num_pass_plays_with_blitzers = len(pass_plays_with_blitzers)
pass_plays_total = len(pass_plays)
pass_blitz_percentage = (num_pass_plays_with_blitzers / pass_plays_total) * 100 if pass_plays_total > 0 else 0
print(f"Total pass plays: {pass_plays_total}")
print(f"Pass plays with at least one blitzer: {num_pass_plays_with_blitzers}")
print(f"Percentage of pass plays with blitzers: {pass_blitz_percentage:.2f}%")

# breakdown of plays by number of blitzers
blitzer_distribution = blitzer_counts_by_play['isLikelyBlitzer'].value_counts().sort_index()
print("\nDistribution of blitzers per play:")
print(blitzer_distribution)

# classify plays as blitzes if > threshold_for_blitz_play likely blitzers
threshold_for_blitz_play = 1
plays_with_significant_blitz = blitzer_counts_by_play[blitzer_counts_by_play['isLikelyBlitzer'] > threshold_for_blitz_play]
print(f"\nPlays with more than {threshold_for_blitz_play} blitzer(s): {len(plays_with_significant_blitz)}")
print(f"Percentage: {(len(plays_with_significant_blitz) / total_plays) * 100:.2f}%")

## Modeling

### Objective

We attempt to predict whether a **defender will blitz** (`isLikelyBlitzer = 1`) based **only on information available before the snap**. In NFL game situations, quarterbacks and offensive coordinators must make real-time protection adjustments based on pre-snap alignments, player movement, and game context. To mirror this reality, our model is strictly limited to **pre-snap observable features**.

---

## Feature Selection

Our features are selected based on what a quarterback or coach could realistically observe live:

- **Tracking Data (Pre-Snap, `tracking_week_*`)**:
  - `x`, `y`: Player's location on the field at the snap.
  - `s`: Player speed (yards/second) — shows if a defender is creeping forward.
  - `a`: Player acceleration (yards/second²) — sudden movement may indicate blitz.
  - `dis`: Distance traveled since the prior frame.
  - `o`: Player orientation (which direction their body is facing).
  - `dir`: Direction of player motion.

- **Play Metadata (`plays.csv`)**:
  - `down`: Current down (1st, 2nd, 3rd, 4th) — blitzes more common on late downs.
  - `yardsToGo`: Distance to first down — long distances can encourage blitzing.
  - `playClockAtSnap`: Remaining play clock time — last-second snaps may alter defensive behavior.
  - `offenseFormation`: Offensive alignment (categorical).
  - `pff_passCoverage`: Defensive coverage scheme (categorical).
  - `pff_manZone`: Whether the defense was playing man or zone.

- **Player Metadata (`players.csv`)**:
  - `position`: Player position group (e.g., LB, CB, S, DE).

All categorical variables are **one-hot encoded** to integrate cleanly with our modeling approach.

By using only pre-snap tracking and contextual information, the model realistically approximates how a quarterback or coach would scout for potential blitzers in real time.

---

## Preprocessing Steps

1. **Filter to defenders only**: Identify players on the defensive team (`club != possessionTeam`).
2. **Restrict frames** to `frameType == 'SNAP'`: Capture exact alignment and motion at the moment of the snap.
3. **Merge metadata**: Add player position and play context (down, distance, formation, coverage).
4. **Handle missing values**: Fill missing categorical fields (e.g., `offenseFormation`) with "Unknown".
5. **One-hot encode** categorical features: Transform `position`, `offenseFormation`, `pff_passCoverage`, and `pff_manZone`.
6. **Prepare X and y**:
   - **X**: Pre-snap physical and contextual features.
   - **y**: Binary label (`isLikelyBlitzer`), engineered from motion and distance heuristics.

---

## Baseline Model

- **Model:** Logistic Regression
- **Class Imbalance Handling:** 
  - Apply `class_weight='balanced'` to automatically adjust for the minority class (blitzers).
  - This prevents the model from simply predicting "no blitz" every time, and encourages sensitivity to rare blitz behaviors.
- **Train-Test Split:** 
  - 80% of data used for training, 20% held out for testing.
  - Stratified sampling is used to preserve the blitz/non-blitz ratio.

Rishi, we should merge these into master_df, unless that would make compute time intractable.

In [ ]:
# merge in play-level context if not already there
if 'possessionTeam' not in snap_tracking.columns:
    snap_tracking = snap_tracking.merge(plays_df[['gameId', 'playId', 'possessionTeam']], on=['gameId', 'playId'], how='left')

# filter to defenders
snap_tracking_defense = snap_tracking[snap_tracking['club'] != snap_tracking['possessionTeam']].copy()

# merge player positions
if 'position' not in snap_tracking_defense.columns:
    snap_tracking_defense = snap_tracking_defense.merge(players_df[['nflId', 'position']], on='nflId', how='left')

# merge play metadata
metadata_cols = ['gameId', 'playId', 'down', 'yardsToGo', 'offenseFormation', 'playClockAtSnap', 'pff_passCoverage', 'pff_manZone']
if not all(col in snap_tracking_defense.columns for col in ['down', 'yardsToGo', 'offenseFormation']):
    snap_tracking_defense = snap_tracking_defense.merge(plays_df[metadata_cols], on=['gameId', 'playId'], how='left')

I didn't run code, so explicitly check for missingness before imputing NaNs.

In [ ]:
# fill all missing categorical fields
categorical_cols = []
for col in snap_tracking_defense.select_dtypes(include=['object']).columns:
    if col not in ['gameId', 'playId', 'nflId', 'displayName', 'position', 'playDirection', 'offenseFormation', 'pff_passCoverage', 'pff_manZone']:
        categorical_cols.append(col)
for col in categorical_cols:
    snap_tracking_defense[col] = snap_tracking_defense[col].fillna('Unknown')

In [ ]:
# one-hot encode categorical columns
snap_tracking_encoded = pd.get_dummies(
    snap_tracking_defense,
    columns=['position', 'offenseFormation', 'pff_passCoverage', 'pff_manZone'],
    drop_first=True
)

In [ ]:
X = snap_tracking_encoded.drop(columns=['gameId', 'playId', 'nflId', 'isLikelyBlitzer'])
# drop columns that are strings
for col in X.select_dtypes(include=['object']).columns:
    if col not in ['gameId', 'playId', 'nflId']:
        X = X.drop(columns=[col])
y = snap_tracking_encoded['isLikelyBlitzer']
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [ ]:
pipeline = make_pipeline(
    SimpleImputer(strategy='mean'),           
    LogisticRegression(max_iter=1000,
                       class_weight='balanced')
)

In [ ]:
# logistic regression
logreg = pipeline.fit(X_train, y_train)

In [ ]:
# evaluate results
y_pred = logreg.predict(X_test)
print("Classification Report (Logistic Regression Baseline):")
print(classification_report(y_test, y_pred))

In [ ]:
# test on a few plays
